In [3]:
import cv2
import numpy as np
import json
from matplotlib import pyplot as plt
import os
from utils import *

In [4]:
with open('saved_regions.json', 'r') as file:
    data = json.load(file)

In [21]:
data.keys()

dict_keys(['ct.jpg', 'ct2.jpg', 'ct3.jpg', 'scopy.jpg', 'scopy2.jpg', 'scopy3.jpg', 'x_ray.png'])

In [32]:
for img_name in data.keys():
    img = cv2.imread('sample_imgs/' + img_name)
    
    for rc in data[img_name]: # read selected regions
        x,y,w,h = rc
        src = img[y:y+h, x:x+w]
        
        try:
            src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        except:
            pass

        imgs = {}
        imgs['origin'] = src

        dst = normalzie_linear(src)
        imgs['normal_linear'] = dst
        imgs['normal_linear_INV'] = 255-dst
        
        dst = cv2.equalizeHist(src)
        imgs['hist_eqaulize'] = dst
        imgs['hist_eqaulize_INV'] = 255-dst
        hist_equ = dst
        
        clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8, 8))
        dst = clahe.apply(src)
        imgs['clahe'] = dst
        imgs['clahe_INV'] = 255-dst
        
        gaussian = cv2.GaussianBlur(src, (9, 9), 10.0)
        dst = cv2.addWeighted(src, 1.5, gaussian, -0.5, 0)
        imgs['unsharped_mask'] = dst

        dst = cv2.bilateralFilter(src, -1, 10, 5)
        imgs['edge_filter'] = dst
        imgs['edge_filter_INV'] = 255-dst

        dst = cv2.bilateralFilter(hist_equ, -1, 20, 5)
        imgs['hist equ and edge filter'] = dst
        imgs['hist equ and edge filter_INV'] = 255-dst

        heq = imgs['clahe']
        dst = cv2.bilateralFilter(heq, -1, 40, 5)
        imgs['clahe and edge_filter'] = dst

        kernel = np.ones((4, 4), np.uint8)
        dst = cv2.dilate(src, kernel, iterations=1)
        imgs['morp_or'] = dst
        
        kernel = np.ones((5, 5), np.uint8)
        dst = cv2.dilate(imgs['clahe'], kernel, iterations=1)
        imgs['clahe and morp_or'] = dst
        
        dst = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY,29,2)
        imgs['adaptive_gaussian_29'] = dst
        imgs['adaptive_gaussian_29_INV'] = 255-dst

        # 이거를 먼저 전처리 하는게 좋을 듯
        _, dst = cv2.threshold(src,110, 255, cv2.THRESH_TOZERO)
        imgs['tozero_110'] = dst
        
        for key in imgs.keys():
            s_img = imgs[key]
            file_path = 'Outputs/normalization_outputs/'
            cv2.imwrite(file_path + str(rc) + key + '.jpg',s_img)

src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

imgs = {}

imgs['origin'] = src


dst = normalzie_linear(src)
imgs['normal_linear'] = dst
imgs['normal_linear_INV'] = 255-dst


dst = cv2.equalizeHist(src)
imgs['hist_eqaulize'] = dst
imgs['hist_eqaulize_INV'] = 255-dst
hist_equ = dst


clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8, 8))
dst = clahe.apply(src)
imgs['clahe'] = dst
imgs['clahe_INV'] = 255-dst


gaussian = cv2.GaussianBlur(src, (9, 9), 10.0)
dst = cv2.addWeighted(src, 1.5, gaussian, -0.5, 0)
imgs['unsharped_mask'] = dst

dst = cv2.bilateralFilter(src, -1, 10, 5)
imgs['edge_filter'] = dst
imgs['edge_filter_INV'] = 255-dst

dst = cv2.bilateralFilter(hist_equ, -1, 20, 5)
imgs['hist equ and edge filter'] = dst
imgs['hist equ and edge filter_INV'] = 255-dst

heq = imgs['clahe']
dst = cv2.bilateralFilter(heq, -1, 40, 5)
imgs['clahe and edge_filter'] = dst

kernel = np.ones((4, 4), np.uint8)
dst = cv2.dilate(src, kernel, iterations=1)
imgs['morp_or'] = dst


kernel = np.ones((5, 5), np.uint8)
dst = cv2.dilate(imgs['clahe'], kernel, iterations=1)
imgs['clahe and morp_or'] = dst


dst = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
cv2.THRESH_BINARY,29,2)
imgs['adaptive_gaussian_29'] = dst
imgs['adaptive_gaussian_29_INV'] = 255-dst



# 이거를 먼저 전처리 하는게 좋을 듯
_, dst = cv2.threshold(src,110, 255, cv2.THRESH_TOZERO)
imgs['tozero_110'] = dst


for key in imgs.keys():
    img = imgs[key]
    file_path = 'Outputs/normalization_outputs/'
    cv2.imwrite(file_path + str(rc) + key + '.jpg',img)